## 사전설정

In [31]:
%cd /home/ubuntu/kr-img2latex

/home/ubuntu/kr-img2latex


In [32]:
pip install -U imbalanced-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install --upgrade tensorflow-hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [34]:
import torch
import torch.nn as nn
import torch.optim as optim

!pip install --upgrade transformers
from transformers import BertTokenizer, DonutProcessor
from transformers import BertTokenizerFast, BertModel


from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import re
import json
import glob, shutil
import xml.etree.ElementTree as elemTree
from collections import Counter
from itertools import chain
from pathlib import Path
from typing import List, Dict, Union, Tuple, Any


from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

from datasets import Dataset

!pip install kiwipiepy
from kiwipiepy import Kiwi, Match
kiwi = Kiwi()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [35]:
LATEX_START = "<latex>"
LATEX_END = "</latex>"

new_tokens = [
    LATEX_START,
    LATEX_END
]

## 데이터 로드

In [36]:
train_json_files = glob.glob('./data/train/annotations/*')

In [37]:
test_json_files = glob.glob('./data/test/annotations/*')

In [76]:
def get_gt_string(files):
    ground_truths = []
    labels = []

    for f in tqdm(files):
        filename = os.path.basename(f)
        filepath = Path(f)
        
        
        with open(filepath,encoding='utf-8') as fp:
            data = json.load(fp)
            
        label = filename.split('_')[2]
        data_series = data['segments']

        all_lines = []

        NON_OBJECT = False
        

        for d_line in data_series:
            if 'equation' not in d_line:
                NON_OBJECT = True
                break

            if '$' not in d_line['equation']:
                new_line = d_line['equation']
            else:
                equation = d_line['equation'].split('$')
                latex_line = equation[0]
                for i, e in enumerate(equation[1:]):
                    if i%2 == 0:
                        latex_line += LATEX_START + e
                    else:
                        latex_line += LATEX_END + e

                new_line = latex_line
        
        all_lines.append(new_line)

        if not NON_OBJECT:
            ground_truths += all_lines
            labels += [label] * len(all_lines)
        else:
            pass

    return ground_truths, labels

ground_truths, gt_labels = get_gt_string(train_json_files)


100%|██████████| 150450/150450 [00:06<00:00, 24900.30it/s]


In [39]:
print(ground_truths[:10])
print(gt_labels[:10])

['<latex>36+24+20+16+4=100 \\%</latex>', '<latex>y=-x+3+m</latex>.', '<latex>\\displaystyle 5</latex> <latex>\\displaystyle \\frac{1}{2},\\frac{1}{5}</latex>', '<latex>\\frac{100}{3600}+20</latex>', '5', '<latex>-1<2 \\cos x \\leq 1 \\quad,-\\frac{1}{2}<\\cos x \\leq \\frac{1}{2}</latex>', '<latex>68=1X68</latex>', '쪽 쪽', '<latex>-3\\cfrac{1}{3}</latex>을 수직선에 그려보면', '파 <latex>\\displaystyle \\sigma (X)=\\sqrt{V(X)}= \\;  \\sqrt{11}</latex>']
['e6', 'm2', 'e4', 'm1', 'h1', 'h3', 'm1', 'e4', 'm1', 'h3']


In [82]:
len(ground_truths)

127060

In [83]:
len(gt_labels)

127060

In [40]:
labels = [
    'e4','e5','e6',
    'm1','m2','m3',
    'h1','h2','h3'
]

id2label = {}
label2id = {}

for i, item in enumerate(labels):
    id2label[i] = item
    label2id[item] = i

### processor

In [42]:
#메인 모델 토크나이저
processor = DonutProcessor.from_pretrained("./pretrained/main/epoch-2, batch-2, img_size-480, max_length-560,copy/processor")
tokenizer = processor.tokenizer
tokenizer_name='math_ocr'

### 키위파이 토크나이저, 메인 모델 토크나이저
한글 정제 : 키위파이 토크나이저 <br>
라텍스 정제 : 메인 모델 토크나이저

In [85]:
def korean_tokenizer(texts):
    
    from kiwipiepy.utils import Stopwords
    stopwords = Stopwords()
    
    add_words=['연립부등식','시속','유리함수','교점','밑변','부분합','쌓기나무','사인법칙','피타고라스','평행이동','대분수','도수분포표','이등변삼각형','상대도수','최솟값','중근','표본공간','닮음비',
              '유한소수','연립부등식','공통범위','각뿔대','극한값','아랫변','수선의발','허근','중근','근과계수의관계','최단거리','교점','홀수항','짝수항','실근','부분집합',
               '진리집합','공비','직사각형','예각삼각형','직각삼각형','일반항','부분집합','부등호','음의정수','항등함수','근의공식','수직이등분선','원주율','입체도형','준식','최댓값',
               '증가량','감소량','일대일대응','대칭이동','음함수','좌변','우변','외심','조임정리','원주각','원순열','초항','점사이의거리','우함수','극댓값','내림차순','중심각','경우의수','기준량',
               '대응각','수직이등분선','사잇값정리','접선의방정식','모눈칸','입체도형','표준형','변화량','벤다이어그램','최고차항','좌표평전','불연속','수직이등분선','도수분포표','이항','다항함수',
               '원주율','연립부등식','정적분','소인수분해','일반항','최고항','상대도수','배반사건','증가함수','접선의기울기','삼수선','공통인수','무리식','일대일함수','닫힌구간','열린구간','기함수',
               '진분수','호의길이','나머지정리','켤레복소수','무게중심','원순열','평균변화율','곱셈공식','이산확률','대응각','대등점','좌표평면','중복조합','차항계수']
    
    stop_words=['정답','개','학생','답','가능','의','라','가','을','은','과','장','철수','무승부','결과','축구','이란','열량','선택','권',
                '동생','여유','테이프','곳','연주','한라산','현주','소금물','금요일','평가','하인','왼쪽','정확','사용','목걸이','여학생','영지',
                '과자','종이','사진','출전','동안','성윤','탈','양궁','무','통장','자전거','개인','고등학교','되어','입장료','바깥','이때','감',
                '시합','인정','도중','에','계절','이므로','옷','야구','를','밖','므','연필','오리','솟값','처음','동전','안','컵','텐트','만약',
                '태준','역사책','어머니','사과','구슬','선생','만족','이용','식량','깃발','오렌지','화요일','반대','인','나','오른쪽','번','아치고',
                '필요','여름','여야','거짓','장마','부분','은행','직','희연','톱니','주머니','서현','청소년','준','고','고르고','못','퀴','최고',
                '풀','모','밀가루','후','메뚜기','월요일','하루','찰흙','정도','지현이','움','우유','기','서','떡','진아','샌드위치','모둠','시',
                '승부','미생물','꽃','올해','단백질','색종이','입','작년','체인','머지','수박','매듭','자취','재경','달리기','팀','인형','초록색',
                '무','이동','줄','제외','여학생','오','버스','추리','마찬가지','발사','세수','선수','조금','습관','회원','껌','영어','수면','하인',
                '눈','사용','영어','임시','양말','바둑돌','각인','회원','잡지','독서량','대회','어린이','진아','르','남학생','여학생','아래','동쪽',
                '어린이','상자','돈','학생','구슬','우유','서','해결','연습','발','맛','와','학교','공차','학기','조각','림','케이스','저금통','시행',
                '하루','반','마을','위쪽','포도','설명','박테리아','부분','사탕','개수','회','도달','병','도','학교','아버지','영희','가지','아들',
                '시작','방법','다음','대신','학업','키','과일','주스','포도','번호','각도기','시행','사탕','생각','후','다','의자','시계',
                '모두','뒤','떡','연습','자리','톱니','어린이','다','학교','명','그릇','라','물','림','제비','학업','설명','다음','대신','몸무게',
                '아래','은','이번','서','연습','배','작년','시작','동화책','생각','앞','제외','하루','학생','여학생','차','가지','방법','포도','와',
                '형','우유','떡','진아','샌드위치','모둠','시','승부','그림','재범']

    nums=['일','이','삼','사','오','육','칠','팔','구','십','십일','십이','이십']

    for num in nums:
        variant_words=['{}차방정식'.format(num),'정{}각형'.format(num),'{}각기둥'.format(num),'{}각형'.format(num),
                       '{}차함수'.format(num),'{}차곡선'.format(num),'{}각뿔'.format(num),'{}차부등식'.format(num),
                       '{}차식'.format(num)]
        
        for word in variant_words:
            add_words.append(word)

    for word in add_words:
        kiwi.add_user_word(word,'NNG')

    for word in stop_words:
        stopwords.add((word, 'NNG'))

    masks={'부분 집합':'부분집합','근과 계수와의 관계':'근과계수의관계','꼭짓점의 개수':'꼭짓점개수','다항 함수':'다항함수','접선의 방정식':'접선의방정식',
           '닫힌 구간':'닫힌구간','열린 구간':'열린구간','이등변 삼각형':'이등변삼각형','나무쌓기':'쌓기나무','산술과 기하평균':'산술기하평균','산술 기하평균':'산술기하평균',
           '각 기둥':'각기둥','차항의 계수':'차항계수','경우의 수':'경우의수','최대값':'최댓값','점 사이의 거리':'점사이의거리','사잇값 정리':'사잇값정리','사잇값의 정리':'사잇값정리','소인수 분해':'소인수분해',
           '수선의 발':'수선의발','최단 거리':'최단거리','부분 집합':'부분집합','근의 공식':'근의공식','접선의 방정식':'접선의방정식','호의 길이':'호의길이','나머지 정리':'나머지 정리',
           '접선의 기울기':'접선의기울기'}

    # ground_truths 토크나이징 후 1차 정제 
    uncleaned_ground_truths=[]
    for text in texts:
        print('원문','-',text)
        text=text.replace('\\displaystyle','')
        latexs=re.findall('<latex>.*?</latex>',text) # latex 부분만 추출
        real_text=re.sub('<latex>.*?</latex>',"",text) # 텍스트 부분만 추출
        
        for mask in masks.keys():
            
            if mask in real_text: # text 부분에 mask dictonary 내의 key-->value 형태로 전환
                
                real_text=re.sub(mask,masks[mask],real_text)
        
        print('1차 가공','-',real_text)
        
        text_tokens=kiwi.tokenize(real_text,stopwords=stopwords)
                
        text_nn_tokens=[]
        cleaned_latex_tokens=[]
                
        for t in text_tokens:
            # 텍스트에서 명사만 추출
            if t.tag=='NNG' or t.tag=='NNP':
                text_nn_tokens.append(t.form)

        if len(latexs)>=1: # 라텍스가 있는 부분에서만 작동
            for latex in latexs:

                latex_tokens=tokenizer.tokenize(latex)
                latex_tokens=[x for x in latex_tokens if x.startswith('\\')] # \\sin,\\cos 등 필요한 기호만 필터링
    

                if len(latex_tokens)>=1:  
                    cl_la_tk=' '.join(latex_tokens) # 라텍스를 한 문장으로 합침
                    cleaned_latex_tokens.append(cl_la_tk)

                else:
                    pass

        cleaned_text=' '.join(text_nn_tokens)
        cleaned_latex=' '.join(cleaned_latex_tokens)
        ct_cl=cleaned_text+cleaned_latex
        uncleaned_ground_truths.append(ct_cl)
        #print('가공 완료','-',ct_cl)
        #print('----------------------------------------')

    
    return uncleaned_ground_truths

In [109]:
def cleansing_texts_and_labels(texts):
    
    # 불용어 제거로 인한 sequence 길이 조절
    
    uncleaned_ground_truths=[]
    uncleaned_labels=[]

    tmp_gt_dicts={'e4':[],'e5':[],'e6':[],
                  'm1':[],'m2':[],'m3':[],
                  'h1':[],'h2':[],'h3':[]}

    for i,v in tqdm(enumerate(texts)):
        
        # 해당 문장이 12자 이상이라면 그냥 패스
        
        if len(v)<=12:
            tmp_gt_dicts[gt_labels[i]].append(v)
            
        # 해당 문장이 12자 미만이라면 임의의 해당 레이블의 딕셔너리로 매핑되어 리스트 형태로 들어감
        
        else:
            uncleaned_ground_truths.append(v)
            uncleaned_labels.append(gt_labels[i])

    for k,v in tmp_gt_dicts.items():
        
        # 기본 3 단어씩 붙여서 하나의 sequence화
        
        while len(v)>=3:
            
            if len(v)>=3:
                long_sent=r' '.join(v[:3])
                v=v[3:]
                uncleaned_ground_truths.append(long_sent)
                uncleaned_labels.append(k)
                
            # 해당 딕셔너리에 단어가 3 단어 미만이라면 남은 단어들을 모두 join
            
            elif 1<=len(v)<=2:
                long_sent=r' '.join(v)
                v=[]
                uncleaned_ground_truths.append(long_sent)
                uncleaned_labels.append(k)

    cleaned_ground_truths=[]
    cleaned_labels=[]

    for i,v in enumerate(tmp_ground_truths):
        v=v.replace('\\','')
        if len(v)>=2:
            cleaned_ground_truths.append(v)
            cleaned_labels.append(gt_labels[i])
    
    return cleaned_ground_truths,cleaned_labels

cleaned_ground_truths,cleaned_labels=cleansing_texts_and_labels(uncleaned_ground_truths)

127060it [00:00, 2397985.36it/s]


In [92]:
def check_for_unk(texts):
    ids = tokenizer(texts).input_ids
    tokens = [tokenizer.tokenize(x) for x in texts]

    unk_tokens = []
    for example_ids, example_tokens in zip(ids, tokens):
        example_unk_tokens = []
        for i in range(len(example_ids)):
            if example_ids[i] == tokenizer.unk_token_id:
                try:
                    example_unk_tokens.append(example_tokens[i])
                except:
                    pass

        unk_tokens += example_unk_tokens

    return unk_tokens

unk_tokens = check_for_unk(cleaned_ground_truths)

In [93]:
text_unk_tokens = list(set(unk_tokens))
print(len(text_unk_tokens))

0


In [128]:
import tensorflow as tf 
import tensorflow_hub as hub 
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.disable_v2_behavior()

In [160]:
def encoding(x):
    numid=label2id[x]
    return numid

num_split=int(len(cleaned_labels)*0.8)

y_encoding = list(map(encoding,cleaned_labels))
y_one_hot=tf.one_hot(y_encoding, depth=9)

with tf.compat.v1.Session() as sess:
    y_one_hot_np = sess.run(y_one_hot)

# 데이터를 훈련 데이터와 테스트 데이터로 분할
num_split = int(len(cleaned_labels) * 0.8)
X_train = np.asarray(cleaned_ground_truths[:num_split])
y_train = np.asarray(y_one_hot_np[:num_split])
X_test = np.asarray(cleaned_ground_truths[num_split:])
y_test = np.asarray(y_one_hot_np[num_split:])

In [163]:
y_train.shape

(62240, 9)

In [145]:
X_train.shape

(62240,)

In [ ]:
labels_one_hot = tf.one_hot(labels, depth=9)

In [121]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

RuntimeError: variable_scope module_2/ was unused but the corresponding name_scope was already taken.

In [133]:
from cachetools import TTLCache,cached
cache = TTLCache(maxsize=100, ttl=21600)

@cached(cache)
def ELMoEmbedding(x):
    return elmo(tf.reshape(tf.cast(x,tf.string),[-1]),signature='default',as_dict=True)['elmo']

def create_model_architecture():
    input_text = tf.keras.layers.Input(shape=(1,), dtype='string',name='input_text')
    embedding = tf.keras.layers.Lambda(ELMoEmbedding,output_shape = (1024,), name = 'elmo_embedding')(input_text)

    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024,return_sequences=False,recurrent_dropout=0.2,dropout=0.2,name="BiLSTM"))(embedding)
    x = tf.keras.layers.Dense(512,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    #x = embedding
    x = tf.keras.layers.Dense(256,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(9,activation='softmax')(x)


    model = tf.keras.Model(inputs = [input_text],outputs = predictions)
    
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['categorical_accuracy'])

    return model

In [148]:
model = create_model_architecture()
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_text (InputLayer)     [(None, 1)]               0         
                                                                 
 elmo_embedding (Lambda)     (None, None, 1024)        0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 2048)              16785408  
 onal)                                                           
                                                                 
 dense_3 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                           

In [167]:
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    sess.run(tf.compat.v1.tables_initializer())
    # history = model.fit(np.asarray(df_train.clean_tweet),y_train,epochs=5,batch_size = 2,validation_split=0.2)
    history = model.fit(X_train,y_train,epochs=3,batch_size = 32,validation_split=0.2)
    model.save_weights('./pretrained/Sub/elmo-model')

Train on 49792 samples, validate on 12448 samples
Epoch 1/3
49792/49792 [==============================] - ETA: 0s - loss: 1.6891 - acc: 0.3647

2023-08-06 15:05:19.059037: W tensorflow/c/c_api.cc:304] Operation '{name:'loss_5/mul' id:16856 op device:{requested: '', assigned: ''} def:{{{node loss_5/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_5/mul/x, loss_5/dense_11_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


49792/49792 [==============================] - 676s 14ms/sample - loss: 1.6891 - acc: 0.3647 - val_loss: 1.5064 - val_acc: 0.4296
Epoch 2/3
49792/49792 [==============================] - 671s 13ms/sample - loss: 1.4977 - acc: 0.4332 - val_loss: 1.4288 - val_acc: 0.4645
Epoch 3/3
49792/49792 [==============================] - 678s 14ms/sample - loss: 1.4247 - acc: 0.4645 - val_loss: 1.3833 - val_acc: 0.4860
